# Регрессия: прогноз CC50

Построение моделей для предсказания CC50 по химическим дескрипторам.

In [19]:
# Подготовка данных
X = df[features]
y = df["CC50, mM"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Линейная модель
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

# Обучение остальных моделей
rf_grid.fit(X_train, y_train)
y_pred_rf = rf_grid.predict(X_test)

xgb_grid.fit(X_train, y_train)
y_pred_xgb = xgb_grid.predict(X_test)

lgb_grid.fit(X_train, y_train)
y_pred_lgb = lgb_grid.predict(X_test)

# Оценка
for name, preds in [("Linear Regression", y_pred_lr), ("Random Forest", y_pred_rf),
                    ("XGBoost", y_pred_xgb), ("LightGBM", y_pred_lgb)]:
    print(f"{name}: R2={r2_score(y_test, preds):.3f}, MAE={mean_absolute_error(y_test, preds):.2f}, MSE={mean_squared_error(y_test, preds):.2f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002118 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15084
[LightGBM] [Info] Number of data points in the train set: 638, number of used features: 166
[LightGBM] [Info] Start training from score 572.483446
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

# Регрессия: прогноз токсичности (CC50)

В этом разделе выполняются следующие шаги:

1. **Формирование задачи и подготовка данных**  
   - Целевая переменная — CC50 (мМ).  
   - Признаки — все дескрипторы `features`, без IC50 и SI.  
   - Сплит данных: 80% для обучения, 20% для теста (`train_test_split`).  
   - Масштабирование признаков через `StandardScaler` для моделей, чувствительных к шкале.

2. **Обучение и настройка моделей**  
   - **LinearRegression** (без подбора гиперпараметров).  
   - **RandomForestRegressor** (GridSearchCV по `n_estimators=[100,300]`, `max_depth=[None,10]`).  
   - **XGBRegressor** (GridSearchCV по `n_estimators=[100,300]`, `max_depth=[3,6]`).  
   - **LGBMRegressor** (GridSearchCV по `n_estimators=[100,300]`, `max_depth=[-1,10]`).  
   - Кросс-валидация 5-fold, метрика оптимизации — R² (scoring='r2').

3. **Результаты на тестовой выборке**

| Модель               | R²          | MAE     | MSE             |
|----------------------|-------------|---------|-----------------|
| Linear Regression    | –36 381.655 | 9 675.62 | 17 139 052 271.22 |
| Random Forest        | 0.453       | 300.00  | 257 465.54      |
| XGBoost              | 0.468       | 293.52  | 250 627.19      |
| LightGBM             | 0.452       | 295.58  | 258 046.12      |

4. **Анализ результатов**  
   - **LinearRegression** показал катастрофически низкое качество (отрицательное R²), т.к. простая линейная зависимость не подходит для прогнозирования токсичности.  
   - **Ансамбли деревьев** (Random Forest, XGBoost, LightGBM) позволили объяснить ≈45–47% дисперсии CC50.  
   - **XGBoost** продемонстрировал наилучшее качество по R² (0.468) и минимальную MAE (~293 мМ).  
   - Разрыв между MSE и MAE указывает на наличие редких, но сильных выбросов CC50 в данных.

> **Вывод:** для задачи регрессии CC50 подходят нелинейные ансамблевые методы, при этом XGBoost даёт наилучший баланс точности и устойчивости к выбросам. Линейная регрессия без преобразований неэффективна.


In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import warnings
warnings.filterwarnings("ignore")

# Загрузка и подготовка данных
df = pd.read_excel("Данные_для_курсовои_Классическое_МО.xlsx")
df.drop(columns=["Unnamed: 0"], inplace=True)
constant_cols = [col for col in df.columns if df[col].nunique() == 1]
df.drop(columns=constant_cols, inplace=True)
df.dropna(inplace=True)
features = [col for col in df.columns if col not in ["IC50, mM", "CC50, mM", "SI"]]
scaler = StandardScaler()
